In [1]:
import ee
import geetools

In [2]:
# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()


In [3]:
S2collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

In [4]:
S2collection

In [5]:
start_date = '2022-02-01'
end_date = '2023-10-31'

In [6]:
def maskS2clouds(image):
    qa = image.select('QA60')

#    Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

# Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

In [54]:
poi = ee.Geometry.Point(-3.920372, 56.147340) # centre of airthrey loch

In [58]:
minimumLon = -3.925116
minimumLat = 56.144342
maximumLon = -3.912912
maximumLat = 56.148563
roi = ee.Geometry.Rectangle(minimumLon, minimumLat, maximumLon, maximumLat)

In [59]:
S2_airth = S2collection.filterDate(start_date, end_date).filterBounds(roi)#.map(maskS2clouds)

In [62]:
nimg = S2_airth.toList(S2_airth.size().getInfo()).size().getInfo()            
                                  
for i in range(nimg):
    img = ee.Image(S2_airth.toList(nimg).get(i))
#     date = img.date().format('yyyy-MM-dd').getInfo()
    index = img.get('system:index').getInfo()

    task = ee.batch.Export.image.toDrive(img.toFloat(), 
                                          folder='GEE_Images_S2_NEWWWWW',
                                          description=index,
                                          fileNamePrefix= index,
                                          region = roi,
                                          scale=10,
                                          #dimensions = (256,256), 
                                          # fileFormat = 'TFRecord',
                                          maxPixels = 1e10)
    task.start()


In [ ]:
# this is working, but I'm not able to run these through acolite processing
# wonder if the file format is slightly off, or maybe there is some metadata missing?
# wonder if I really need the toFloat() thingy?